---
### Tugas Submission Dicoding Machine Learning Terapan Sistem Rekomendasi
- **Nama  :** Hafidh Soekma Ardiansyah <br>
- **Asal  :** Pasuruan, Pandaan, Jawa Timur <br>
- **Email :** hapoed123@gmail.com <br>
- **Dataset :** https://www.kaggle.com/vishalmane109/anime-recommendations-database
---

# Data Loading

## Import Library 
Kita akan menuliskan beberapa library yang akan kita butuhkan nanti.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import files, output
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

print("Semua library yang dibutuhkan telah terhubung")

Semua library yang dibutuhkan telah terhubung


## Hubungkan Colab ke Kaggle
Kita akan menghubungkan colab dengan kaggle, untuk mendownload datasets yang akan kita butuhkan. Dengan mengimport file kaggle.json yang tersedia di situs kaggle teman masing-masing.

In [2]:
!rm kaggle.json
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

output.clear()
print("Colab telah terhubung dengan kaggle")

Colab telah terhubung dengan kaggle


## Download dataset
Kita download datasets yang akan kita butuhkan.

In [3]:
!rm anime-recommendations-database.zip
!kaggle datasets download -d vishalmane109/anime-recommendations-database

output.clear()
print("Datasets telah berhasil didownload")

Datasets telah berhasil didownload


## Extract dataset
Kita proses datasets tersebut supaya bisa digunakan di colab.

In [4]:
!rm Anime_data.csv
!unzip anime-recommendations-database.zip
!rm anime-recommendations-database.zip

output.clear()
print("Datasets telah siap digunakan")

Datasets telah siap digunakan


## Buka dataset csv dengan pandas
Kita buka datasets tersebut dan kita tampilkan 5 data teratas.

In [5]:
df = pd.read_csv("./Anime_data.csv")
df.head()

,Anime_id,Title,Genre,Synopsis,Type,Producer,Studio,Rating,ScoredBy,Popularity,Members,Episodes,Source,Aired,Link
0,1,Cowboy Bebop,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...","In the year 2071, humanity has colonized sever...",TV,['Bandai Visual'],['Sunrise'],8.81,363889.0,39.0,704490.0,26.0,Original,"Apr 3, 1998 to Apr 24, 1999",https://myanimelist.net/anime/1/Cowboy_Bebop
1,5,Cowboy Bebop: Tengoku no Tobira,"['Action', 'Space', 'Drama', 'Mystery', 'Sci-Fi']","Another day, another bounty—such is the life o...",Movie,"['Sunrise', 'Bandai Visual']",['Bones'],8.41,111187.0,475.0,179899.0,1.0,Original,"Sep 1, 2001",https://myanimelist.net/anime/5/Cowboy_Bebop__...
2,6,Trigun,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...","Vash the Stampede is the man with a $$60,000,0...",TV,['Victor Entertainment'],['Madhouse'],8.31,197451.0,158.0,372709.0,26.0,Manga,"Apr 1, 1998 to Sep 30, 1998",https://myanimelist.net/anime/6/Trigun
3,7,Witch Hunter Robin,"['Action', 'Magic', 'Police', 'Supernatural', ...",Witches are individuals with special powers li...,TV,['Bandai Visual'],['Sunrise'],7.34,31875.0,1278.0,74889.0,26.0,Original,"Jul 2, 2002 to Dec 24, 2002",https://myanimelist.net/anime/7/Witch_Hunter_R...
4,8,Bouken Ou Beet,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",It is the dark century and the people are suff...,TV,NaN,['Toei Animation'],7.04,4757.0,3968.0,11247.0,52.0,Manga,"Sep 30, 2004 to Sep 29, 2005",https://myanimelist.net/anime/8/Bouken_Ou_Beet


## Data unik pada setiap kolom
Kita lihat jumlah data unik pada setiap kolom

In [6]:
for column in df.columns:
    print(f"Jumlah data unik pada kolom {column}:", len(df[column].unique()))

Jumlah data unik pada kolom Anime_id: 16850
Jumlah data unik pada kolom Title: 16737
Jumlah data unik pada kolom Genre: 4578
Jumlah data unik pada kolom Synopsis: 15195
Jumlah data unik pada kolom Type: 8
Jumlah data unik pada kolom Producer: 3006
Jumlah data unik pada kolom Studio: 778
Jumlah data unik pada kolom Rating: 627
Jumlah data unik pada kolom ScoredBy: 5454
Jumlah data unik pada kolom Popularity: 14383
Jumlah data unik pada kolom Members: 8044
Jumlah data unik pada kolom Episodes: 197
Jumlah data unik pada kolom Source: 17
Jumlah data unik pada kolom Aired: 11128
Jumlah data unik pada kolom Link: 16217


# Exploratory Data Analysis

## Deskripsi Variabel
Setelah itu, kita tampilkan deskripsi variabel pada datasets tersebut. Saya rubah semua nama kolom menjadi kecil supaya memudahkan kita untuk menggunakan kolom tersebut.

In [7]:
df.columns = [column.lower() for column in df.columns]
print("Total data:", df.shape[0], end="\n\n")
df.info()

Total data: 17002

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17002 entries, 0 to 17001
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   anime_id    17002 non-null  int64  
 1   title       17002 non-null  object 
 2   genre       14990 non-null  object 
 3   synopsis    15583 non-null  object 
 4   type        16368 non-null  object 
 5   producer    7635 non-null   object 
 6   studio      7919 non-null   object 
 7   rating      14425 non-null  float64
 8   scoredby    13227 non-null  float64
 9   popularity  16368 non-null  float64
 10  members     17002 non-null  float64
 11  episodes    14085 non-null  float64
 12  source      15075 non-null  object 
 13  aired       16368 non-null  object 
 14  link        16368 non-null  object 
dtypes: float64(5), int64(1), object(9)
memory usage: 1.9+ MB


## Deskripsi Statistik
Setelah itu, kita tampilkan deskripsi statistik pada data tersebut.

In [8]:
df.describe()

,anime_id,rating,scoredby,popularity,members,episodes
count,17002.000000,14425.000000,1.322700e+04,16368.000000,1.700200e+04,14085.000000
mean,20446.579638,6.287867,1.139084e+04,8131.919599,2.038130e+04,11.482712
std,14342.513259,1.141401,4.328434e+04,4714.683351,7.121404e+04,44.089040
min,1.000000,1.000000,1.000000e+00,1.000000,0.000000e+00,1.000000
25%,5581.500000,5.620000,4.300000e+01,4042.500000,1.450000e+02,1.000000
50%,21334.000000,6.410000,4.780000e+02,8115.000000,1.113000e+03,1.000000
75%,34789.250000,7.090000,3.831000e+03,12208.250000,7.855750e+03,12.000000
max,40960.000000,10.000000,1.006242e+06,16338.000000,1.451708e+06,1818.000000


## Univariate Analysis


### Categorical Features
Kita analisa untuk kolom categorical.

In [9]:
categorical_features = ["title", "genre", "synopsis", "type",
                        "producer", "studio", "source", "aired",
                        "link"]
for column in categorical_features:
    print(f"Banyak data pada kolom '{column}':", len(df[column].unique()))
    print(f"Data unik pada kolom '{column}':", df[column].unique(), end="\n\n")

Banyak data pada kolom 'title': 16737
Data unik pada kolom 'title': ['Cowboy Bebop' 'Cowboy Bebop: Tengoku no Tobira' 'Trigun' ...
 'X Bomber Pilot' 'Ganko-chan' 'Original C-V-P Momoko']

Banyak data pada kolom 'genre': 4578
Data unik pada kolom 'genre': ["['Action', 'Adventure', 'Comedy', 'Drama', 'Sci-Fi', 'Space']"
 "['Action', 'Space', 'Drama', 'Mystery', 'Sci-Fi']"
 "['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'Drama', 'Shounen']" ...
 "['Action', 'Adventure', 'Comedy', 'Historical', 'Seinen']"
 "['Comedy', 'Game', 'School']"
 "['Action', 'Fantasy', 'Horror', 'Martial Arts', 'Samurai']"]

Banyak data pada kolom 'synopsis': 15195
Data unik pada kolom 'synopsis': ['In the year 2071, humanity has colonized several of the planets and moons of the solar system leaving the now uninhabitable surface of planet Earth behind. The Inter Solar System Police attempts to keep peace in the galaxy, aided in part by outlaw bounty hunters, referred to as "Cowboys." The ragtag team aboard the spaces

### Numerical Features
Kita analisa untuk kolom numerical.

In [10]:
numerical_features = ["anime_id", "rating", "scoredby", "popularity", 
                      "members", "episodes"]
for column in numerical_features:
    print(f"\nJumlah data pada kolom '{column}':", len(df[column].unique()))
    print(df[[column]].describe())


Jumlah data pada kolom 'anime_id': 16850
           anime_id
count  17002.000000
mean   20446.579638
std    14342.513259
min        1.000000
25%     5581.500000
50%    21334.000000
75%    34789.250000
max    40960.000000

Jumlah data pada kolom 'rating': 627
             rating
count  14425.000000
mean       6.287867
std        1.141401
min        1.000000
25%        5.620000
50%        6.410000
75%        7.090000
max       10.000000

Jumlah data pada kolom 'scoredby': 5454
           scoredby
count  1.322700e+04
mean   1.139084e+04
std    4.328434e+04
min    1.000000e+00
25%    4.300000e+01
50%    4.780000e+02
75%    3.831000e+03
max    1.006242e+06

Jumlah data pada kolom 'popularity': 14383
         popularity
count  16368.000000
mean    8131.919599
std     4714.683351
min        1.000000
25%     4042.500000
50%     8115.000000
75%    12208.250000
max    16338.000000

Jumlah data pada kolom 'members': 8044
            members
count  1.700200e+04
mean   2.038130e+04
std    7.121404

# Data Preparation

## Menangani Missing Value

### Cek Missing Value
Setelah itu kita cek data yang kosong.

In [11]:
df.isna().sum()

anime_id         0
title            0
genre         2012
synopsis      1419
type           634
producer      9367
studio        9083
rating        2577
scoredby      3775
popularity     634
members          0
episodes      2917
source        1927
aired          634
link           634
dtype: int64

### Hapus Missing Value
Kita akan menghapus data yang kosong

In [12]:
df = df.dropna()
df.isna().sum()

anime_id      0
title         0
genre         0
synopsis      0
type          0
producer      0
studio        0
rating        0
scoredby      0
popularity    0
members       0
episodes      0
source        0
aired         0
link          0
dtype: int64

## Hapus semua data yang sama
Setelah itu kita hapus data yang sama. Supaya model machine learning kita mempelajari banyak variasi data bukan hanya data yang sama saja.

In [13]:
print("Total data sebelum hapus duplicates:", df.shape[0])
df = df.drop_duplicates()
print("Total data sebelum hapus duplicates:", df.shape[0])

Total data sebelum hapus duplicates: 4479
Total data sebelum hapus duplicates: 4459


# Model Development

## Content Based Filtering

### Buat Model
Kita buat model content based filtering dengan menggunakan TF-IDF Vectorizer

In [14]:
tf = TfidfVectorizer()
tf.fit(df['genre'])
tf.get_feature_names() 

['action',
 'adventure',
 'ai',
 'arts',
 'cars',
 'comedy',
 'dementia',
 'demons',
 'drama',
 'ecchi',
 'fantasy',
 'fi',
 'game',
 'harem',
 'hentai',
 'historical',
 'horror',
 'josei',
 'kids',
 'life',
 'magic',
 'martial',
 'mecha',
 'military',
 'music',
 'mystery',
 'of',
 'parody',
 'police',
 'power',
 'psychological',
 'romance',
 'samurai',
 'school',
 'sci',
 'seinen',
 'shoujo',
 'shounen',
 'slice',
 'space',
 'sports',
 'super',
 'supernatural',
 'thriller',
 'vampire',
 'yaoi',
 'yuri']

### Latih Model
Kita latih model content based filtering kita

In [15]:
tfidf_matrix = tf.fit_transform(df['genre'])      
print("Ukuran matrix:", tfidf_matrix.shape) 
tfidf_matrix.todense()

Ukuran matrix: (4459, 47)


matrix([[0.29690062, 0.35874628, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.28628532, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.33234705, 0.40157634, 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.49543606, 0.59863749, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

### Buat Derajat Kesamaan
Kita akan menghitung derajat kesamaan antar setiap anime

In [16]:
cosine_sim = cosine_similarity(tfidf_matrix) 
cosine_sim_df = pd.DataFrame(cosine_sim, index=df['title'], columns=df['title'])
print('Ukuran Cosine:', cosine_sim_df.shape)
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

Ukuran Cosine: (4459, 4459)


title,Uchuu Koukyoushi Maetel: Ginga Tetsudou 999 Gaiden,Mikakunin de Shinkoukei,Shining Hearts: Shiawase no Pan Specials,Romeo x Juliet,"Kamen no Maid Guy: Ano Natsu, Ichiban Yutaka na Chichi."
title,,,,,
Gunslinger Stratos The Animation,0.506349,0.000000,0.000000,0.000000,0.240606
Rerere no Tensai Bakabon,0.000000,0.269369,0.000000,0.000000,0.403057
Tennis no Ouji-sama: Pair Prince,0.182562,0.084780,0.000000,0.188743,0.126857
Nanatsu no Bitoku Specials,0.000000,0.000000,0.585473,0.255224,0.636223
Getsuyoubi no Tawawa,0.000000,0.634600,0.000000,0.000000,0.442452
Yokoyama Mitsuteru Sangokushi,0.000000,0.000000,0.000000,0.482940,0.198553
Medama Oyaji no April Fools',0.000000,0.000000,0.000000,0.000000,0.000000
Dousou Kai Again,0.000000,0.000000,0.000000,0.000000,0.000000
Sol Bianca,0.777837,0.000000,0.000000,0.000000,0.155271


### Buat Fungsi Rekomendasi
Kita akan membuat fungsi untuk mengambil rekomendasi

In [17]:
def anime_recommendations(judul_anime, similarity_data=cosine_sim_df, items=df[['title', 'genre']], k=5):
    index = similarity_data.loc[:,judul_anime].to_numpy().argpartition(
        range(-1, -k, -1))
    
    closest = similarity_data.columns[index[-1:-(k+2):-1]]
    closest = closest.drop(judul_anime, errors='ignore')
    return pd.DataFrame(closest).merge(items).head(k)

print("Fungsi sudah dibuat")

Fungsi sudah dibuat


### Test Model
Kita akan testing model kita

In [18]:
anime_recommendations("Bleach", k=10)

,title,genre
0,Zettai Karen Children: The Unlimited - Hyoubu ...,"['Action', 'Super Power', 'Supernatural', 'Sho..."
1,Hunter x Hunter,"['Action', 'Adventure', 'Shounen', 'Super Power']"
2,Hunter x Hunter Movie 1: Phantom Rouge,"['Action', 'Adventure', 'Super Power', 'Shounen']"
3,Hunter x Hunter Movie 2: The Last Mission,"['Action', 'Adventure', 'Super Power', 'Shounen']"
4,Hunter x Hunter (2011),"['Action', 'Adventure', 'Super Power', 'Shounen']"
5,Kekkai Sensen: Ousama no Restaurant no Ousama,"['Action', 'Comedy', 'Super Power', 'Supernatu..."
6,Code:Breaker,"['Action', 'Comedy', 'School', 'Shounen', 'Sup..."
7,One Piece: Adventure of Nebulandia,"['Action', 'Adventure', 'Comedy', 'Super Power..."
8,One Piece Movie 2: Nejimaki-jima no Daibouken,"['Action', 'Adventure', 'Comedy', 'Super Power..."
9,One Piece: Glorious Island,"['Action', 'Adventure', 'Comedy', 'Super Power..."


### Evaluation Model
Setelah itu kita akan melihat hasil model tersebut dengan menggunakan precision evaluation

In [93]:
def get_precision(anime_title, k=5):
    score_precision = 0
    real_result = df[df["title"].eq(anime_title)]
    real_result = real_result["genre"].values[0]
    real_result = real_result.replace("'", "")
    real_result = real_result.replace("[", "")
    real_result = real_result.replace("]", "")
    real_result = real_result.split(", ")

    anime_result = anime_recommendations(anime_title, k=k)
    for i in range(k):
        genre_anime = anime_result.iloc[i, 1]
        genre_anime = genre_anime.replace("'", "")
        genre_anime = genre_anime.replace("[", "")
        genre_anime = genre_anime.replace("]", "")
        genre_anime = genre_anime.split(", ")
        result_genre = set(real_result) - set(genre_anime)
        if len(result_genre) < len(real_result):
            score_precision += 1

    return score_precision/k

testing_anime = ["Sword Art Online", "Bleach", "Naruto"]
evaluation = pd.DataFrame(columns=['precision'], index=testing_anime)
for anime in testing_anime:
    evaluation.loc[anime, "precision"] = get_precision(anime, k=50)

evaluation

,precision
Sword Art Online,1
Bleach,1
Naruto,1
